In [1]:
import numpy as np # linear algebra
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/trained_50k_46/transformers/default/1/__huggingface_repos__.json
/kaggle/input/trained_50k_46/transformers/default/1/qwen_marathi/best-checkpoint-step/config.json
/kaggle/input/trained_50k_46/transformers/default/1/qwen_marathi/best-checkpoint-step/merges.txt
/kaggle/input/trained_50k_46/transformers/default/1/qwen_marathi/best-checkpoint-step/tokenizer.json
/kaggle/input/trained_50k_46/transformers/default/1/qwen_marathi/best-checkpoint-step/vocab.json
/kaggle/input/trained_50k_46/transformers/default/1/qwen_marathi/best-checkpoint-step/tokenizer_config.json
/kaggle/input/trained_50k_46/transformers/default/1/qwen_marathi/best-checkpoint-step/chat_template.jinja
/kaggle/input/trained_50k_46/transformers/default/1/qwen_marathi/best-checkpoint-step/model.safetensors
/kaggle/input/trained_50k_46/transformers/default/1/qwen_marathi/best-checkpoint-step/special_tokens_map.json
/kaggle/input/trained_50k_46/transformers/default/1/qwen_marathi/best-checkpoint-step/added_tokens.j

In [2]:
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == 

In [3]:
!pip install -q sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.2 MB/s eta 0:00:00


In [4]:
!pip install -q wandb

In [5]:
# imports
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding, get_scheduler
from datasets import load_dataset
import torch
import wandb
import torch.nn as nn
from torch.utils.data import DataLoader
from accelerate import Accelerator
from transformers.optimization import AdafactorSchedule
from transformers.optimization import get_cosine_schedule_with_warmup
from accelerate import Accelerator
from tqdm.auto import tqdm
import os
import math
from transformers.optimization import Adafactor, get_scheduler
import evaluate
import shutil

2025-07-13 18:53:13.661140: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752432793.855963      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752432793.919766      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
# checking GPU availability
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Number of GPUs: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")


CUDA available: True
Number of GPUs: 1
GPU 0: Tesla P100-PCIE-16GB


In [7]:
# connect wandb library for graphs
from kaggle_secrets import UserSecretsClient

secret = UserSecretsClient()
os.environ["WANDB_API_KEY"] = secret.get_secret("WANDB_KEY") 

wandb.login()   

wandb: Currently logged in as: 22cs3033 (22cs3033-rgipt-jais) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [8]:
# loading model and tokenizer into the RAM
model_name = "Qwen/Qwen3-0.6B"
model = AutoModelForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True )
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [9]:
# create the response on default pretrained model
prompt = "Dengue cases on the rise in city"
messages = [
    {"role": "system", "content": "You are a translator. who knows marathai and english language very well. Your task is convert given english sentence in marathi."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


In [10]:
response

'<think>\nOkay, the user wants to translate the sentence "Dengue cases on the rise in city" into Marathi. Let me start by breaking down the sentence. \n\nFirst, "Dengue cases" – I know that in Marathi, "dengue" is often transliterated as "देंगु" or sometimes written as "देंगु" with the letter "इ". But since it\'s a specific disease, maybe it\'s better to keep it as "देंगु" for accuracy.\n\nNext, "cases on the rise" – "cases" here refers to the number of cases, so "मामले" is the correct term. "On the rise" can be translated as "उपचार बरावे" or "उपचार बरावे बरावे". Since "on the rise" is more like increasing, "बरावे बरावे" might be better.\n\nThen, "in city" – "महानदर्शन" or "महानदर्शन" is a common term for a city in Marathi. So putting it all together: "देंगु मामले बरावे बरावे महानदर्शन बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे बरावे

In [11]:
# load the english to marathi translation data from huggingface
ds = load_dataset("anujsahani01/English-Marathi")

README.md:   0%|          | 0.00/206 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/621M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/243M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2637962 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/879321 [00:00<?, ? examples/s]

In [12]:
ds

DatasetDict({
    train: Dataset({
        features: ['english', 'marathi'],
        num_rows: 2637962
    })
    test: Dataset({
        features: ['english', 'marathi'],
        num_rows: 879321
    })
})

In [13]:
ds['train'][10]

{'english': 'The funds collected are used for social purposes.',
 'marathi': 'त्या निधीचा वापर सामाजिक कार्यासाठी करतो.'}

In [14]:
# Dataset is too big so slice it accroding your computation resources
df_train = ds["train"].select(range(10000))
df_validation = ds["train"].select(range(10000 , 11000))
df_test = ds["test"].select(range(500))

In [15]:
# Qwen3-0.6B is Causal Language Models so we have finetuned it Instruction-output format. (instruction‑tuning)
def add_instruction(example):
    example["instruction"] = "Convert the English text into Marathi language."
    return example


df_train = df_train.map(add_instruction, batched=False)
df_validation = df_validation.map(add_instruction, batched=False)
df_test = df_test.map(add_instruction, batched=False)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [16]:
ds["train"].column_names

['english', 'marathi']

In [17]:
df_train[0]

{'english': 'Next few months are really crucial for us.',
 'marathi': 'पुढील तीन महिने आमच्यासाठी खूप महत्त्वपूर्ण आहेत.',
 'instruction': 'Convert the English text into Marathi language.'}

In [18]:
# Know about how tokenizer works
inputs = tokenizer('Next few months are really crucial for us.','पुढील तीन महिने आमच्यासाठी खूप महत्त्वपूर्ण आहेत.', 'Convert the English text into Marathi language.' )
inputs

{'input_ids': [5847, 2421, 3951, 525, 2167, 16587, 369, 601, 13, 86162, 72653, 149269, 43647, 91811, 14925, 97, 43647, 60096, 91217, 93948, 42311, 101, 34370, 14925, 228, 87244, 146113, 30484, 107, 31411, 116, 31411, 254, 43647, 14925, 244, 12619, 224, 86162, 91217, 93948, 79238, 30484, 97, 30484, 113, 86162, 12619, 224, 44179, 30484, 96, 14925, 228, 93948, 54784, 97, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [12012, 279, 6364, 1467, 1119, 2876, 66531, 4128, 13]}

In [19]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['Next',
 'Ġfew',
 'Ġmonths',
 'Ġare',
 'Ġreally',
 'Ġcrucial',
 'Ġfor',
 'Ġus',
 '.',
 'à¤ª',
 'à¥ģ',
 'à¤¢',
 'à¥Ģ',
 'à¤²',
 'Ġà¤',
 '¤',
 'à¥Ģ',
 'à¤¨',
 'Ġà¤®',
 'à¤¹',
 'à¤¿à¤',
 '¨',
 'à¥ĩ',
 'Ġà¤',
 'Ĩ',
 'à¤®',
 'à¤ļ',
 'à¥įà¤',
 '¯',
 'à¤¾à¤',
 '¸',
 'à¤¾à¤',
 'ł',
 'à¥Ģ',
 'Ġà¤',
 'ĸ',
 'à¥',
 'Ĥ',
 'à¤ª',
 'Ġà¤®',
 'à¤¹',
 'à¤¤',
 'à¥įà¤',
 '¤',
 'à¥įà¤',
 'µ',
 'à¤ª',
 'à¥',
 'Ĥ',
 'à¤°',
 'à¥įà¤',
 '£',
 'Ġà¤',
 'Ĩ',
 'à¤¹',
 'à¥ĩà¤',
 '¤',
 '.']

In [20]:
instruction = "Convert the English text into Marathi language."
src_text     = "Next few months are really crucial for us."
tgt_text     = "पुढील तीन महिने आमच्यासाठी खूप महत्त्वपूर्ण आहेत."

# Prompt the model should *see*
prompt = f"{instruction} English: {src_text} Marathi:"
print(prompt)

inputs  = tokenizer(prompt, return_tensors="pt") 
print('Length of tokenize form of prompt:', inputs['input_ids'].size())


# Every token sequence in a causal‐LM fine‑tuning setup needs a clear “stop” signal so the model knows when it’s done generating. That’s exactly what tokenizer.eos_token_id is for

labels  = tokenizer(tgt_text, add_special_tokens=False).input_ids + [tokenizer.eos_token_id] #Here labes are tokenize form our marathi output

Convert the English text into Marathi language. English: Next few months are really crucial for us. Marathi:
Length of tokenize form of prompt: torch.Size([1, 23])


In [21]:
inputs

{'input_ids': tensor([[12012,   279,  6364,  1467,  1119,  2876, 66531,  4128,    13,  6364,
            25,  9295,  2421,  3951,   525,  2167, 16587,   369,   601,    13,
          2876, 66531,    25]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [22]:
# The length of tokenize input prompt( instrution + context) is 23

In [23]:
tokenizer.eos_token_id

151645

In [24]:
labels 

[86162,
 72653,
 149269,
 43647,
 91811,
 14925,
 97,
 43647,
 60096,
 91217,
 93948,
 42311,
 101,
 34370,
 14925,
 228,
 87244,
 146113,
 30484,
 107,
 31411,
 116,
 31411,
 254,
 43647,
 14925,
 244,
 12619,
 224,
 86162,
 91217,
 93948,
 79238,
 30484,
 97,
 30484,
 113,
 86162,
 12619,
 224,
 44179,
 30484,
 96,
 14925,
 228,
 93948,
 54784,
 97,
 13,
 151645]

In [25]:
len(labels) # 49 orignal + 1 tokenizer.eos_token_id = 50

50

In [26]:
# For a causal model, you feed it the entire sequence (prompt and target), because it learns by shifting that sequence one token at a time.
model_inputs = tokenizer(prompt + " " + tgt_text + tokenizer.eos_token,  return_tensors="pt")
print('Length of tokenize form of model_inputs:', model_inputs['input_ids'].size())  

# You need to know how many tokens correspond just to the instruction+source (the “prompt”) so that you can later mask them out in the labels.
prompt_len = len(tokenizer(prompt, add_special_tokens=False).input_ids)
print("Tokenized Prompt length:" , prompt_len)

#  Clone so you don’t overwrite the original inputs
labels = model_inputs["input_ids"].clone()


# For every position in the prompt, set label to -100  
# In Hugging Face Trainer, any label token = –100 is ignored by the loss function.
labels[:, :prompt_len] = -100        # works for a batch of size 1 or N
# print('Length that going to be masked' , len(labels))
model_inputs["labels"] = labels

Length of tokenize form of model_inputs: torch.Size([1, 73])
Tokenized Prompt length: 23


In [27]:
labels

tensor([[  -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,  83636,  72653, 149269,  43647,
          91811,  14925,     97,  43647,  60096,  91217,  93948,  42311,    101,
          34370,  14925,    228,  87244, 146113,  30484,    107,  31411,    116,
          31411,    254,  43647,  14925,    244,  12619,    224,  86162,  91217,
          93948,  79238,  30484,     97,  30484,    113,  86162,  12619,    224,
          44179,  30484,     96,  14925,    228,  93948,  54784,     97,     13,
         151645]])

In [28]:
labels.size()

torch.Size([1, 73])

In [29]:
x = labels[0] 
num_ignored = (x == -100).sum().item()
num_not_ignored = (x != -100).sum().item()
print(num_ignored)
print(num_not_ignored)

23
50


In [30]:
model_inputs

{'input_ids': tensor([[ 12012,    279,   6364,   1467,   1119,   2876,  66531,   4128,     13,
           6364,     25,   9295,   2421,   3951,    525,   2167,  16587,    369,
            601,     13,   2876,  66531,     25,  83636,  72653, 149269,  43647,
          91811,  14925,     97,  43647,  60096,  91217,  93948,  42311,    101,
          34370,  14925,    228,  87244, 146113,  30484,    107,  31411,    116,
          31411,    254,  43647,  14925,    244,  12619,    224,  86162,  91217,
          93948,  79238,  30484,     97,  30484,    113,  86162,  12619,    224,
          44179,  30484,     96,  14925,    228,  93948,  54784,     97,     13,
         151645]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]]), 'labels': tensor([[  -100,   -100,   -100,

In [31]:
model_inputs['input_ids'].size()

torch.Size([1, 73])

In [32]:
# this function convert string into vectors
def tokenize_function(examples):

    batch_input_ids      = []
    batch_attention_mask = []
    batch_labels         = []

    # Iterate over the batch
    for instruction, english, marathi in zip(
            examples["instruction"], examples["english"], examples["marathi"]):

        # 1. Build the prompt shown to the model (not trained on)
        prompt_text = f"{instruction} English: {english} Marathi:"
        prompt_ids  = tokenizer(prompt_text, add_special_tokens=False).input_ids

        # 2. Full text = prompt + target + EOS
        full_text = f"{prompt_text} {marathi}{tokenizer.eos_token}"
        tok = tokenizer(full_text, add_special_tokens=False)

        # 3. Create labels: mask the prompt tokens with –100
        label_ids = [-100] * len(prompt_ids) + tok["input_ids"][len(prompt_ids):]

        # 4. Collect results for this example
        batch_input_ids.append(tok["input_ids"])
        batch_attention_mask.append(tok["attention_mask"])
        batch_labels.append(label_ids)

    return {
        "input_ids":      batch_input_ids,
        "attention_mask": batch_attention_mask,
        "labels":         batch_labels,
    }

In [33]:
train_tokenised_ds = df_train.map(tokenize_function, batched=True , num_proc=4 , remove_columns=df_train.column_names)
test_tokenised_ds = df_test.map(tokenize_function, batched=True , num_proc=4 , remove_columns=df_test.column_names)
validation_tokenised_ds = df_validation.map(tokenize_function, batched=True , num_proc=4, remove_columns=df_validation.column_names )

Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/500 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [34]:
test_tokenised_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 500
})

In [35]:
len(test_tokenised_ds[0]['input_ids'])

90

In [36]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})
    model.resize_token_embeddings(len(tokenizer))          # update embeddings
model.config.pad_token_id = tokenizer.pad_token_id

In [37]:
for split in (train_tokenised_ds, test_tokenised_ds, validation_tokenised_ds):
    split.set_format(type="torch")

In [38]:
from transformers import DataCollatorForSeq2Seq

class EfficientDataCollatorForSeq2Seq(DataCollatorForSeq2Seq):
    def __call__(self, features, return_tensors=None):
        # Let the parent class handle padding, returning NumPy arrays.
        batch = super().__call__(features, return_tensors="np")
        # Manually convert each padded NumPy array to a PyTorch tensor.
        tensor_batch = {}
        for key, value in batch.items():
            tensor_batch[key] = torch.from_numpy(value)
        return tensor_batch

data_collator = EfficientDataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    label_pad_token_id=-100,
    pad_to_multiple_of=8,
)

In [39]:
train_tokenised_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})

In [40]:
# Create data loaders
train_dataloader = DataLoader(
        train_tokenised_ds,
        batch_size=2,
        shuffle=True,
        collate_fn=data_collator,
        pin_memory=False
)
    
eval_dataloader = DataLoader(
    validation_tokenised_ds,
    batch_size=2,
    shuffle=False,
    collate_fn=data_collator,
    pin_memory=False
)

test_dataloader = DataLoader(
    test_tokenised_ds,
    batch_size=2,
    shuffle=False,
    collate_fn=data_collator,
    pin_memory=False
)

In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f">>> Using device: {device}")

model.to(device)

>>> Using device: cuda


Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [42]:
#training paramter and Optimizer , scheduler
num_epochs = 1
per_device_train_batch_size = 6
per_device_eval_batch_size = 4
gradient_accumulation_steps = 2
learning_rate = 5e-5
weight_decay = 0.01
warmup_steps = 200
logging_steps = 500
eval_steps = 500
save_total_limit = 2
output_dir = "qwen_marathi"
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
max_train_steps = num_epochs * num_update_steps_per_epoch
optimizer_Adafactor = Adafactor(
    model.parameters(),
    lr=learning_rate,  # Adafactor can compute its own LR, which is often recommended
    scale_parameter=False,
    relative_step=False, # Set to True to use internal learning rate
    warmup_init=False,
    weight_decay=weight_decay,
)
optimizer_AdamW = torch.optim.AdamW(model.parameters(),lr=learning_rate,weight_decay=weight_decay,betas=(0.9, 0.999),eps=1e-8)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer_Adafactor,
    num_warmup_steps=0,
    num_training_steps=max_train_steps,
)


In [43]:

# --- 5. Print Training Setup ---
total_batch_size = per_device_train_batch_size * gradient_accumulation_steps
print("***** Running training *****")
print(f"  Num examples              = {len(train_tokenised_ds)}")
print(f"  Num Epochs                = {num_epochs}")
print(f"  Batch size per device     = {per_device_train_batch_size}")
print(f"  Total train batch size    = {total_batch_size}")
print(f"  Gradient Accumulation     = {gradient_accumulation_steps}")
print(f"  Total optimization steps  = {max_train_steps}")
print(f"  Number of devices         = 1")

torch.cuda.empty_cache()

***** Running training *****
  Num examples              = 10000
  Num Epochs                = 1
  Batch size per device     = 6
  Total train batch size    = 12
  Gradient Accumulation     = 2
  Total optimization steps  = 2500
  Number of devices         = 1


In [44]:
# 8. Initialize WandB (main process only)

wandb.init(
    project="qwen-marathi-finetuning",
    name="qwen-marathi-run2",
    config={
            "learning_rate": learning_rate,
            "num_epochs": num_epochs,
            "batch_size_per_device": per_device_train_batch_size,
            "gradient_accumulation_steps": gradient_accumulation_steps,
            "weight_decay": weight_decay,
            "warmup_steps": warmup_steps,
            "optimizer": optimizer_Adafactor ,
            "lr_scheduler": lr_scheduler,
            "num_gpus": torch.cuda.device_count(),
    }
)


In [45]:
def save_checkpoint(model, tokenizer, output_dir, checkpoint_name, saved_checkpoints, save_total_limit):
    """Saves model and tokenizer, and manages the total number of checkpoints."""
    checkpoint_dir = os.path.join(output_dir, checkpoint_name)
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Save model and tokenizer
    model.save_pretrained(checkpoint_dir)
    tokenizer.save_pretrained(checkpoint_dir)
    print(f"Checkpoint saved to {checkpoint_dir}")
    saved_checkpoints.append(checkpoint_dir)

    # Remove old checkpoints if the limit is exceeded
    if len(saved_checkpoints) > save_total_limit:
        oldest_checkpoint = saved_checkpoints.pop(0)
        # Avoid deleting special checkpoints like 'best' or 'final'
        if "best" not in oldest_checkpoint and "final" not in oldest_checkpoint:
            try:
                shutil.rmtree(oldest_checkpoint)
                print(f"Removed old checkpoint: {oldest_checkpoint}")
            except OSError as e:
                print(f"Error removing old checkpoint {oldest_checkpoint}: {e}")

In [46]:
def evaluate_model(model, eval_dataloader, device):
    """Evaluates the model and returns the average loss."""
    model.eval()
    total_eval_loss = 0.0
    eval_progress = tqdm(eval_dataloader, desc="Evaluating", leave=False)

    with torch.no_grad():
        for batch in eval_progress:
            # Move batch to the same device as the model
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            total_eval_loss += loss.item()

    avg_eval_loss = total_eval_loss / len(eval_dataloader)
    return avg_eval_loss

In [47]:

global_step = 0
total_loss = 0.0
best_eval_loss = float('inf')
saved_checkpoints = []

progress_bar = tqdm(range(max_train_steps) , desc="Training")
model.zero_grad()

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0

    for step, batch in enumerate(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        
        outputs = model(**batch)
        loss = outputs.loss

        loss = loss / gradient_accumulation_steps
        loss.backward()
                
        # optimizer.step()
        # lr_scheduler.step()
        # optimizer.zero_grad()

        epoch_loss += loss.detach().float().item()
        total_loss += loss.detach().float().item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer_Adafactor.step()
            lr_scheduler.step()
            optimizer_Adafactor.zero_grad()

            global_step += 1
            progress_bar.update(1)
            progress_bar.set_postfix({
                'loss': f'{loss.item() * gradient_accumulation_steps:.4f}', # Show actual loss
                'lr': f'{lr_scheduler.get_last_lr()[0]:.2e}',
                'step': global_step
            })
            
            # logging
            if global_step % logging_steps == 0:
                avg_loss = total_loss / global_step
                current_lr = lr_scheduler.get_last_lr()[0]
                
                print(f"[Step {global_step}] train_loss={avg_loss:.4f}, lr={current_lr:.2e}")
                wandb.log({
                        "train/loss": avg_loss,
                        "train/learning_rate": current_lr,
                        "train/epoch": epoch + (step + 1) / len(train_dataloader),
                        "train/global_step": global_step
                })

            # evaluation
            if global_step % eval_steps == 0:
                eval_loss = evaluate_model(model, eval_dataloader, device )
                
                print(f"[Step {global_step}] eval_loss={eval_loss:.4f}")
                wandb.log({"eval/loss": eval_loss, "eval/global_step": global_step})
                # save best
                if eval_loss < best_eval_loss:
                    best_eval_loss = eval_loss
                    save_checkpoint(
                            model, tokenizer, output_dir,
                            f"best-checkpoint-step",
                            saved_checkpoints, save_total_limit
                    )
            model.train()

    
    # end of epoch
    avg_epoch_loss = epoch_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs} • avg_train_loss={avg_epoch_loss:.4f}")
    wandb.log({"train/epoch_loss": avg_epoch_loss, "train/epoch": epoch+1})

    # checkpoint at epoch end
    save_checkpoint(
            model, tokenizer, output_dir,
            f"epoch-{epoch+1}", saved_checkpoints, save_total_limit
    )

# Final evaluation & cleanup
final_eval_loss = evaluate_model(model, eval_dataloader , device)

print(f"Final evaluation • eval_loss={final_eval_loss:.4f}")
wandb.log({"eval/final_loss": final_eval_loss})

save_checkpoint(
        model, tokenizer,  output_dir,
        "final-checkpoint",
        saved_checkpoints, save_total_limit
)

progress_bar.close()
wandb.finish()
print("Training completed!")

Training:   0%|          | 0/2500 [00:00<?, ?it/s]

[Step 500] train_loss=1.6274, lr=4.00e-05


Evaluating:   0%|          | 0/500 [00:00<?, ?it/s]

[Step 500] eval_loss=1.4148
Checkpoint saved to qwen_marathi/best-checkpoint-step
[Step 1000] train_loss=1.4732, lr=3.00e-05


Evaluating:   0%|          | 0/500 [00:00<?, ?it/s]

...

[Step 2500] eval_loss=0.9989
Checkpoint saved to qwen_marathi/best-checkpoint-step
Epoch 1/1 • avg_train_loss=0.6204
Checkpoint saved to qwen_marathi/epoch-1


Evaluating:   0%|          | 0/500 [00:00<?, ?it/s]

Final evaluation • eval_loss=0.9989
Checkpoint saved to qwen_marathi/final-checkpoint


eval/final_loss,▁
eval/global_step,▁▃▅▆█
eval/loss,█▅▃▂▁
train/epoch,▁▃▅▆██
train/epoch_loss,▁
train/global_step,▁▃▅▆█
train/learning_rate,█▆▅▃▁
train/loss,█▅▃▂▁
eval/final_loss,0.99891
eval/global_step,2500
eval/loss,0.99891


Training completed!


In [48]:
def translate_to_marathi(english_sentence: str) -> str:
    instruction = "Convert the English text into Marathi language."
    prompt = f"{instruction} English: {english_sentence} Marathi:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate with a reasonable max length
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,           # greedy
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )
    # Strip off prompt tokens and decode rest
    generated = outputs[0][inputs.input_ids.size(1):]
    return tokenizer.decode(generated, skip_special_tokens=True)

# Example
print(translate_to_marathi("The objective of this article and the following two articles is to deepen our appreciation for Gods qualities that we may think of less often than his principal attributes."))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 या लेखाच्या विषयावरून आपण देवाच्या विशेष वैशिष्ट्यांचा विचार करू शकतो आणि त्याच्या विशेष वैशिष्ट्यांचा विचार करू शकतो आणि आपण त्याच्या विशेष वैशिष्ट्यांचा विचार करू शकतो आणि त्याच्या विशेष वैशिष्ट्यांचा विचार करू शकतो.


In [49]:
import torch
import evaluate
from tqdm.auto import tqdm

def evaluate_model_bleu(model, tokenizer, df_test, batch_size=8):
    print("Evaluating model and calculating SacreBLEU score...")
    metric = evaluate.load("sacrebleu")
    model.eval()  # Set model to inference mode

    # This is a standard practice for decoder-only models.
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = model.config.eos_token_id

    predictions, refs = [], []

    for i in tqdm(range(0, len(df_test), batch_size), desc="Generating Predictions"):
        batch = df_test[i : i + batch_size]
        src_sentences = batch["english"]
        ref_sentences = batch["marathi"]

        prompts = [
            f"Convert the English text into Marathi language. English: {src} Marathi:"
            for src in src_sentences
        ]

        # Tokenize inputs
        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512,
            padding_side='left',  # Critical for decoder-only models
        ).to(model.device)

        with torch.no_grad():
            generated = model.generate(
                **inputs,
                max_new_tokens=512,
                do_sample=False,  # Use greedy decoding
                eos_token_id=tokenizer.eos_token_id,
            )

        # Efficiently slice and decode the entire batch at once
        prompt_token_length = inputs.input_ids.shape[1]
        generated_only_ids = generated[:, prompt_token_length:]  # Corrected line

        # Use batch_decode for faster processing
        decoded_preds = tokenizer.batch_decode(generated_only_ids, skip_special_tokens=True)

        # Strip any extra whitespace from each prediction
        predictions.extend([pred.strip() for pred in decoded_preds])
        refs.extend([[t] for t in ref_sentences])

    # Compute and return the final BLEU score
    bleu_result = metric.compute(predictions=predictions, references=refs)
    return bleu_result

# Compile model for faster inference
print("Compiling model for faster inference...")
model = torch.compile(model)

# --- How to use the function ---
# Assuming your model, tokenizer, and df_test are loaded
bleu_score = evaluate_model_bleu(model, tokenizer, df_test)
print(f"SacreBLEU on test set: {bleu_score['score']:.2f}")

Compiling model for faster inference...
Evaluating model and calculating SacreBLEU score...


Generating Predictions:   0%|          | 0/63 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p'

SacreBLEU on test set: 2.44


In [50]:
# resume training function

def resume_training(
        resume_from_checkpoint: str,
        num_additional_epochs: int,
        resume_from_epoch: int,
        output_dir: str = "qwen_marathi",
        per_device_train_batch_size: int = 2,
        gradient_accumulation_steps: int = 4,
        learning_rate: float = 5e-5,
        weight_decay: float = 0.01,
        warmup_steps: int = 200,
        logging_steps: int = 500,
        eval_steps: int = 500,
        save_total_limit: int = 2,
        use_wandb: bool = True
    ):
    """
    Resume fine-tuning from a previous checkpoint.
    Assumes the original datasets, dataloaders, and helper functions
    are available in the global scope of the script.
    """

    if not os.path.isdir(resume_from_checkpoint):
        raise FileNotFoundError(f"Checkpoint directory not found: {resume_from_checkpoint}")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f">>> Resuming training from: {resume_from_checkpoint}")
    print(f">>> Additional epochs to train: {num_additional_epochs}")
    print(f">>> Resuming from epoch (for logging): {resume_from_epoch}")


    model = AutoModelForCausalLM.from_pretrained(resume_from_checkpoint)
    tokenizer = AutoTokenizer.from_pretrained(resume_from_checkpoint)
    model.to(device)

    # Ensure pad token is set correctly
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})
        model.resize_token_embeddings(len(tokenizer))
    model.config.pad_token_id = tokenizer.pad_token_id

    optimizer = Adafactor(
        model.parameters(),
        lr=learning_rate,       
        scale_parameter=False,  
        relative_step=False,    
        warmup_init=False,      
        weight_decay=weight_decay,
    )

    # Re-compute number of optimization steps
    num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
    max_train_steps = num_additional_epochs * num_update_steps_per_epoch

    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=max_train_steps,
    )

    if use_wandb:
        import wandb
        wandb.init(
            project="qwen-marathi-finetuning",
            name=f"qwen-marathi-resume-epoch{resume_from_epoch}",
            config={
                "learning_rate": learning_rate,
                "num_additional_epochs": num_additional_epochs,
                "batch_size_per_device": per_device_train_batch_size,
                "gradient_accumulation_steps": gradient_accumulation_steps,
                "weight_decay": weight_decay,
                "warmup_steps": warmup_steps,
                "optimizer": "Adafactor",
                "lr_scheduler": "linear",
                "num_gpus": torch.cuda.device_count(),
                "resume_from_checkpoint": resume_from_checkpoint,
            },
            resume="allow",
        )

    global_step = 0
    total_loss = 0.0
    best_eval_loss = float('inf')
    saved_checkpoints = []

    progress_bar = tqdm(range(max_train_steps), desc="Resuming Training")

    model.zero_grad()

    for epoch in range(num_additional_epochs):
        model.train()
        epoch_loss = 0.0

        for step, batch in enumerate(train_dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}

            outputs = model(**batch)
            loss = outputs.loss
            loss = loss / gradient_accumulation_steps
            loss.backward()

            epoch_loss += loss.detach().float().item()
            total_loss += loss.detach().float().item()

            if (step + 1) % gradient_accumulation_steps == 0:
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

                global_step += 1
                progress_bar.update(1)
                progress_bar.set_postfix({
                    'loss': f'{loss.item() * gradient_accumulation_steps:.4f}',
                    'lr': f'{lr_scheduler.get_last_lr()[0]:.2e}',
                    'step': global_step,
                })

                # Logging
                if global_step % logging_steps == 0 and use_wandb:
                    avg_loss = total_loss / global_step
                    wandb.log({
                        "train/loss": avg_loss,
                        "train/learning_rate": lr_scheduler.get_last_lr()[0],
                        "train/epoch": resume_from_epoch + epoch + (step + 1) / len(train_dataloader),
                        "train/global_step": global_step,
                    })

                # Evaluation
                if global_step % eval_steps == 0:
                    eval_loss = evaluate_model(model, eval_dataloader, device)
                    print(f"[Resume Step {global_step}] eval_loss={eval_loss:.4f}")
                    if use_wandb:
                        wandb.log({"eval/loss": eval_loss, "eval/global_step": global_step})

                    if eval_loss < best_eval_loss:
                        best_eval_loss = eval_loss
                        save_checkpoint(
                            model, tokenizer, output_dir,
                            f"best-resume-checkpoint-step",
                            saved_checkpoints, save_total_limit
                        )
                model.train()

        # End-of-epoch checkpoint
        avg_epoch_loss = epoch_loss / len(train_dataloader)
        print(f"Resume Epoch {resume_from_epoch + epoch + 1} completed • avg_train_loss={avg_epoch_loss:.4f}")
        if use_wandb:
            wandb.log({"train/epoch_loss": avg_epoch_loss, "train/epoch": resume_from_epoch + epoch + 1})

        save_checkpoint(
            model, tokenizer, output_dir,
            f"resume-epoch-{resume_from_epoch + epoch + 1}",
            saved_checkpoints, save_total_limit
        )

    final_eval_loss = evaluate_model(model, eval_dataloader, device)
    print(f"Resume Final evaluation • eval_loss={final_eval_loss:.4f}")
    if use_wandb:
        wandb.log({"eval/final_loss": final_eval_loss})

    save_checkpoint(
        model, tokenizer, output_dir,
        "resume-final-checkpoint",
        saved_checkpoints, save_total_limit
    )

    progress_bar.close()
    if use_wandb:
        wandb.finish()
    print("Resume training completed!")

In [51]:
resume_training(
    resume_from_checkpoint="/kaggle/input/trained_50k_46/transformers/default/1/qwen_marathi/best-checkpoint-step",
    num_additional_epochs=1,
    resume_from_epoch=1
)

>>> Resuming training from: /kaggle/input/trained_50k_46/transformers/default/1/qwen_marathi/best-checkpoint-step
>>> Additional epochs to train: 1
>>> Resuming from epoch (for logging): 1


Resuming Training:   0%|          | 0/1250 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/500 [00:00<?, ?it/s]

[Resume Step 500] eval_loss=0.8555
Checkpoint saved to qwen_marathi/best-resume-checkpoint-step


Evaluating:   0%|          | 0/500 [00:00<?, ?it/s]

[Resume Step 1000] eval_loss=0.7941
Checkpoint saved to qwen_marathi/best-resume-checkpoint-step
Resume Epoch 2 completed • avg_train_loss=0.2029
Checkpoint saved to qwen_marathi/resume-epoch-2


Evaluating:   0%|          | 0/500 [00:00<?, ?it/s]

Resume Final evaluation • eval_loss=0.7775
Checkpoint saved to qwen_marathi/resume-final-checkpoint


eval/final_loss,▁
eval/global_step,▁█
eval/loss,█▁
train/epoch,▁▆█
train/epoch_loss,▁
train/global_step,▁█
train/learning_rate,█▁
train/loss,█▁
eval/final_loss,0.77754
eval/global_step,1000
eval/loss,0.79407


Resume training completed!


In [52]:
# save final model & tokenizer to disk
output_path = "qwen_marathi/final_model"
os.makedirs(output_path, exist_ok=True)

model.save_pretrained(output_path)
tokenizer.save_pretrained(output_path)

print(f"Final model and tokenizer saved to: {output_path}")

Final model and tokenizer saved to: qwen_marathi/final_model


In [55]:
import gc
# Memory cleanup
torch.cuda.empty_cache()
gc.collect()

3205